In [1]:
from langchain.graph_transformers.diffbot import DiffbotNLPGraphTransformer
from langchain.schema import Document

In [2]:
diffbot_nlp = DiffbotNLPGraphTransformer(diffbot_api_key="")

In [3]:
from langchain.graphs import Neo4jGraph

url="bolt://localhost:7687"
username="neo4j"
password="pleaseletmein"

graph = Neo4jGraph(
    url=url,
    username=username, 
    password=password
)

In [4]:
from langchain.document_loaders import WikipediaLoader

# No chunking because
# Maximum of 100,000 characters per document and 1,000,000 total characters per API request.
for query in ["Leonhard Euler", "Barack Obama", "United States", "Neo4j", "Warcraft"]:
    raw_documents = WikipediaLoader(query=query).load()
    graph_documents = diffbot_nlp.transform_documents(raw_documents)
    graph.add_graph_documents(graph_documents)
    print(query)

Leonhard Euler
Barack Obama
United States
Neo4j
Warcraft


In [5]:
graph.refresh_schema()

In [6]:
import os
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI

os.environ['OPENAI_API_KEY'] = "sk-"

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,
)
     

In [12]:
chain.run("Where was Leonhard Euler located?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Leonhard Euler'})-[:ALL_PERSON_LOCATIONS]->(l:Location)
RETURN l.name
Full Context:
[{'l.name': 'Basel'}]

> Finished chain.


'Leonhard Euler was located in Basel.'

In [8]:
chain.run("What is Leonhard Euler field of work?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Leonhard Euler'})-[:FIELD_OF_WORK]->(s:Skill)
RETURN s.name
Full Context:
[{'s.name': 'complex analysis'}, {'s.name': 'analytic number theory'}]

> Finished chain.


"Leonhard Euler's field of work included complex analysis and analytic number theory."

In [9]:
chain.run("Where did Barack Obama attend university?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Barack Obama'})-[:EDUCATED_AT]->(o:Organization)
RETURN o.name
Full Context:
[{'o.name': 'University of Hawaii'}, {'o.name': 'Punahou School'}, {'o.name': 'Columbia University'}, {'o.name': 'Harvard Law School'}, {'o.name': 'University of Chicago Law School'}, {'o.name': 'Maseno National School'}, {'o.name': 'Harvard University'}]

> Finished chain.


'Barack Obama attended the University of Hawaii, Punahou School, Columbia University, Harvard Law School, and the University of Chicago Law School.'